In [1]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env

import rllib_setup

ray.init()
env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }


config = (
    PPOConfig().rollouts(num_rollout_workers=10)
    .training(gamma=0.9, lr=0.01) 
    .resources(num_gpus=1)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
    .environment(disable_env_checking=True)
)
print(config.to_dict())
# Build a Algorithm object from the config and run one training iteration.
algo = config.build(env=env_name)

2023-09-29 13:18:08,144	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:442: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/yuan/ResMan/man/lib/python3.9/site-

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 1, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': True, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_envs_pe

In [2]:
algo.train()

2023-09-29 13:18:20,339	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
2023-09-29 13:18:35,081	WARNING ppo.py:520 -- The mean reward returned from the environment is 380.3554992675781 but the vf_clip_param is set to 10.0. Consider increasing it for policy: Machine_0 to improve value function convergence.


{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'Machine_0': {'learner_stats': {'allreduce_latency': 0.0,
     'grad_gnorm': 6.36249870856603,
     'cur_kl_coeff': 0.20000000000000007,
     'cur_lr': 0.010000000000000002,
     'total_loss': 7.416676092147827,
     'policy_loss': 0.20174345221221873,
     'vf_loss': 6.943181117375691,
     'vf_explained_var': -1.5894571940104166e-08,
     'kl': 1.3587572819863756,
     'entropy': 2.8487657845020293,
     'entropy_coeff': 0.0},
    'model': {},
    'custom_metrics': {},
    'num_agent_steps_trained': 117.0,
    'num_grad_updates_lifetime': 15.5,
    'diff_num_grad_updates_vs_sampler_policy': 14.5},
   'Machine_7': {'learner_stats': {'allreduce_latency': 0.0,
     'grad_gnorm': 4.486259746799866,
     'cur_kl_coeff': 0.20000000000000004,
     'cur_lr': 0.01,
     'total_loss': 9.149518422285716,
     'policy_loss': 0.11943101973518727,
     'vf_loss': 8.788565146923066,
     'vf_explained_var': -1.2914339701334636e-08,


In [3]:
from ray import tune

In [5]:
alg_name = "PPO"
tune.run(
    alg_name,
    name="PPO",
    stop={"episodes_total": 10000},
    checkpoint_freq=10,
    config=config.to_dict(),
)

2023-09-29 13:22:25,157	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-09-29 13:22:25,160	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/a3c.py` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-09-29 13:22:25,363	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/maml/maml.py` has been deprecated. Use `rllib_contrib/maml/` instead. This will raise an error in the future!


(PPO pid=104298) 2023-09-29 13:22:32,438	WARNING algorithm_config.py:643 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=104298) 2023-09-29 13:22:32,457	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PPO pid=104298) 2023-09-29 13:22:42,542	INFO trainable.py:173 -- Trainable.setup took 10.087 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=104298) 2023-09-29 13:22:43,722	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
(PPO pid=104298) 2023-09-29 13:22:56,131	WARNING ppo.py:520 -- The mean reward returned from the environment is 840.9097900390625 but the vf_clip_param is set to 10.0. Consider increasing it for policy: Machine_8 to improve value function convergence.


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_tag,hostname,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
PPO_VJS_2cb97_00000,111890,{},"{'num_env_steps_sampled': 112000, 'num_env_steps_trained': 112000, 'num_agent_steps_sampled': 111890, 'num_agent_steps_trained': 111890}",{},2023-09-29_13-31-09,False,nan,{},nan,nan,nan,0,0,0,lwh-Super-Server,28,192.168.3.6,111890,111890,112000,4000,186.99,112000,4000,186.99,0,10,0,0,4000,"{'cpu_util_percent': 46.55200000000001, 'ram_util_percent': 47.92400000000001, 'gpu_util_percent0': 0.7848, 'vram_util_percent0': 0.8077799479166667, 'gpu_util_percent1': 0.6128, 'vram_util_percent1': 0.8033854166666667}",104298,{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}",502.159,21.4095,502.159,"{'training_iteration_time_ms': 20071.489, 'sample_time_ms': 1223.229, 'learn_time_ms': 18765.439, 'learn_throughput': 213.158, 'synch_weights_time_ms': 79.647}",1695965469,112000,28,2cb97_00000


2023-09-29 13:22:56,851	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 0.622 s, which may be a performance bottleneck.
2023-09-29 13:22:56,854	WARNING util.py:315 -- The `process_trial_result` operation took 0.628 s, which may be a performance bottleneck.
2023-09-29 13:22:56,855	WARNING util.py:315 -- Processing trial results took 0.629 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-09-29 13:22:56,856	WARNING util.py:315 -- The `process_trial_result` operation took 0.630 s, which may be a performance bottleneck.
2023-09-29 13:24:14,010	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 1.065 s, which may be a performance bottleneck.
2023-09-29 13:24:14,018	WARNING util.py:315 -- The `process_trial_result` operation took 1.075 s, which may be a performance bottleneck.
2023-09-29 13:24:14,019	WARNING util.py:315 -- Processing trial results took 1.076 s, which may be a performance bott

TuneError: ('Trials did not complete', [PPO_VJS_2cb97_00000])

In [4]:
for j in range(10):
    for i in range(10):
        info = algo.train()
        info['info']['learner']['Machine_6']['learner_stats']['total_loss']

(raylet) [2023-09-29 11:19:28,025 E 3473473 3473473] (raylet) node_manager.cc:3069: 4 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 9648b9014e4abaede66aee64b1344246a1b2b8bbe34248617d11f84d, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


ValueError: Expected parameter loc (Tensor of shape (46, 1)) of distribution Normal(loc: torch.Size([46, 1]), scale: torch.Size([46, 1])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<SplitBackward0>)
 tracebackTraceback (most recent call last):
  File "/home/yuan/ResM

(raylet) [2023-09-29 11:21:28,027 E 3473473 3473473] (raylet) node_manager.cc:3069: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 9648b9014e4abaede66aee64b1344246a1b2b8bbe34248617d11f84d, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-09-29 11:22:28,028 E 3473473 3473473] (raylet) node_manager.cc:3069: 9 Workers (tasks / actors) kill